# Gstreamer Pipelines 
## von der Kamera zum DNN mit Python 

In diesem Jupyter Notebook findet ihr die Ausarbeitung zur Vorlesung im Wahlpflichtkurs "Advanced Python" an der Beuth Hochschule für Technik. Die Umsetzung ähnlicher Projekte sollhiermit erleichtert werden. 

Jeder Rechner mit Ubuntu 18.04 sollte die gleichen Befehle benötigen. NVIDIA AGX Xavier besitzt eine aarch64 Architektur. Der Guide funktioniert somit auch für Arm basierte Systeme. 

Am Ende des Notebooks versteht ihr den Umgang mit Gstreamer und könnt eine dedizierte Video-Pipeline die durch ein DNN klassifiziert wird erstellen. 

Im verkürzten "Coronasemester" SS20 habe ich leider keine weitere Energie bereit gehabt um dieses Projekt vollständiger abzuschließen. Möchte jedoch die ursprünglich geplante Bildkorrektur von einer Aufnahme im Dämmerlicht zu einer im Tageslicht in den kommenden Wochen zu Ende ausarbeiten. 

Die Gliederung
1. Einrichtung der Hardware
2. Grundlagen von Gstreamer
3. Verbindung zu OpenCV und Einbindung des DNN
4. dynamische Pipelines mit GstInterpipe & GstInference (von RidgeRun)

<img src="pics/day-night.jpg" style="height:1000px;">

## 1 - Einrichtung der Hardware
### 1.1 - Einrichtung der SSH-Verbindung

Ich nutzen den Raspberrry Pi 2 als Bridge, um preiswerte Sensoren nutzen zu können. Der Anschluss geschieht via Cross-Kabel und SSH.

Rasbian und Raspberry OS sollte bereits vorkonfiguriert sein. Folgende Einstellungen sind im Konfigurationsmenü zu aktivieren.
1. Der SSH-Server
2. X11-forwarding (Fensterteilung über ssh)
3. Der CSI Kameraport
4. Um die Leistung zu verbessern, kann man den Pi in die CLI booten.
5. Die automatische Anmeldung kann gewählt werden.
6. Der Default-Name des Pi ist "pi", kann aber angepasst werden

Für jedes genutzte Gerät muss eine eindeutige IP4 Adresse fest vergeben werden. Ich nutze als neues Subnetz 10.0.0.1 für den Receiver und 10.0.0.5 für den Raspberry Pi. DHCP muss abgestellt sein. Moderne Ubuntus bieten in den Netzwerkeinstellungen die Option "Direktverbindung". In älteren Distros nennt es sich "Ad-Hoc" Verbindung.

Nun kann man sich mit dem Befehl "ssh -X pi@<chosen_ip4_address>" als Remote-Terminal, mit Fenster-Weitergabe anmelden.

### 1.2 - Einrichtung des Jetson AGX Xavier mit NVIDIAS Developer Kit

Hierfür benötigt man bei Ersteinrichtung ein zweites Gerät mit Ubuntu 18.04 . Die Installation setzt eine aktuelle Python Umgebung voraus. Man nutzt das kostenfreie NVIDIA Developer Kit zur Installation. Da die Software-Pakete sich noch stark verändern, sollte man von umfassender Personalisierung absehen, man wird das System oft neu aufsetzen.

### 1.3 - Einrichtung der Kamera
Um die unterstützten Auflösungen und Codierung der Kamera aufzulisten kann der Befehl

_v4l2-ctl --list-formats-ext_

verwendet werden. Weitere Befehle sind (X mit 0-63 ersetzen)

_v4l2-compliance -d /dev/videoX -f_ 

_v4l2-ctl -d /dev/videoX --all_

### 1.4 - Vergrößern des UDP-Buffer
Damit mehrere Streams über den gleichen Socket versendet werden können, muss die UDP-Puffer vergrößert werden.
Dazu kann man sich die aktuelle Größe mit den Befehlen

_sysctl net.core.rmem_max_

_sysctl net.core.rmem_default_

ausgeben lassen. Und mit den Befehlen


sudo sysctl -w net.core.rmem_max=8388608

sudo sysctl -w net.core.wmem_max=8388608

auf 8MB erweitern.

### 1.5 Gstreamer installieren
GStreamer ist in der Standardinstallation von Ubuntu enthalten, kann ansonsten aber über das folgende Paket installiert werden:

_sudo apt-get install libgstreamer1.0-0_


Empfehlenswert ist weiterhin noch das Paket:


_sudo apt-get install gstreamer1.0-tools_

## 2 - Grundlagen von Gstreamer

Gstreamer (https://gstreamer.freedesktop.org/) ist Open Source. In Gstreamer bildet man komplexe Pipelines zur  Erstellung von "Streaming Media Apllication".

Eine Gstreamer Pipeline ist modular aufgebaut. Sie wird aus den vielen unterschiedlichen PlugIns zusammen gehängt.
Gstreamer kann in vielen verschiedenen Programmiersprachen bearbeitet werden. 
https://gstreamer.freedesktop.org/bindings/

(Der Standardbaustein eines Muxer oder Demuxer ist im unteren Bild nicht abgebildet.)

<img src="pics/thread-buffering.png" style="width:1000px;">


### 2.1 Kamerastream vom RaspberryPi senden
- Auch die CSI Kamera im Raspberry wird über die Video4Linux2 Treiber angesprochen. 
- Die Kameras liegen dabei im Verzeichnes /dev mit der Bezeichnung video0 bis video63
- Videoformat sowie Chroma (Interlaced YUV 4:2:0) sind Mindestangaben!
- um den Videostrem im "Real Time Streaming Protocol" zu versenden wird die _rtpvrawpay_ Node genutzt.
- die udpsink wandelt den Payload in UPD-konforme Pakete um.
- Wir möchten keinen Sync, das heißt Pakete die nicht verarbeitet werden können werden verworfen.

```bash
gst-launch-1.0 -ev v4l2src \
device=/dev/video0 !      \
'video/x-raw, width=128, height=96,' \
'format=I420, framerate=10/1'!       \
rtpvrawpay !              \
udpsink host=10.0.0.1 port=5000      \
sync=false async=false

```

### 2.2 - Kamerastream empfangen
Auf der Empfängerseite muss der Stream in der umgekehrten Reihenfolge mit den entsprechenden Modulen umgewandelt werden.
Als End-Node wird "xvimagesink" genutzt. die ein X-Fenster öffnet um das Video abzuspielen.

```bash
gst-launch-1.0 udpsrc port=5000 !
'application/x-rtp, media=(string)video,'
'encoding-name=(string)RAW, sampling=YCbCr-4:2:0,'
'width=(string)128, height=(string)96' !
rtpvrawdepay ! xvimagesink
```

Die Informationen der Shell, bei setzen des -ev flag können ebenfalls nötig sein. Zum Beispiel braucht man den timestamp-offset bei der Synchronisation und dem scrollen durch Streams. Die Informationen jedes Moduls werden hierbei aufgeführt.
<img src="pics/pi_gstreamer.png" style="width:1000px;">

## 3 - Gstreamer zu OpenCV
### 3.1 - Gst-Python
Für das aufsetzen des Videostream bietet sich Python an. Hierbei kann man einen Factory Pattern nutzen. 

In [ ]:
import os
import gi
from gi.repository import Gst, GObjekt
GObject.threads_init
Gst.init(None)

class SimplePipeline(object):
    def __init__(self):
        self.pipeline = Gst.Pipeline()
        self.filesrc = Gst.ElementFactory.make('filesrc')
        self.oggdemux = Gst.ElementFactory.make('oggdemux')

        self.pipeline.add(self.filesrc)         # Module dem Bauplan zufügen
        self.pipeline.add(self.oggdemux)
        self.pipeline.add(self.theoradec)
        self.pipeline.add(self.videoconvert)
        self.pipeline.add(self.xvimagesink)

        self.filesrc.link(self.oggdemux)        # Module verlinken
        self.oggdemux.link(self.theoracodec)
        self.theoracodec.link(self.videoconvert)
        self.videoconvert.link(self.xvimagesink)

loop = GObject.MainLoop()

p = SimplePipeline()

p.filesrc.set_property('location', video 'video.ogv')
p.pipeline.set_state(Gst.State.PLAYING)
loop.run

Oder, je nachdem, was man übersichtlicher findet auch string-parsing. Das Debugging größerer Pipelines kann sonst umständlich werden.

In [ ]:
class SimplePipeline(object)
    def __init__(self):
        pipe_desc = ('filesrc name=src !'
                    'oggdemux ! theoradec !'
                    'videoconvert ! xvimagesink')
        self.pipeline = Gst.parse_launch(pipe_desc)
        self-filesrc = self.pipeline.get_by_name('src')

### 3.2 - OpenCV um Videostreams zu nutzen
OpenCV ist zurecht der Standard für Computer Vision. Es beinhaltet über 2500 Module mit absolut aktuellem Bezug.
https://docs.opencv.org/master/d9/df8/tutorial_root.html
Wir nutzen es um die Gstreamer Pipeline zu bearbeiten. Den Videostream anzupassen und durch ein DNN zu bewerten. Die Ergebnisse werden ebenfalls mithilfe von OpenCV herausgestellt.

In [7]:
# Import der benötigten Pakete
from __future__ import print_function
from imutils.video import WebcamVideoStream # https://github.com/jrosebr1/imutils/ für Threads
import cv2                                  # https://github.com/opencv/opencv
import subprocess
import numpy as np
from time import sleep
from queue import Queue


Als erstes laden wir die trainierten Gewichte, und die Erkennungsklassen in OpenCV.
Wir sehen hier im Modellnamen, dass Bilder zur Verarbeitung die Maße 300px x 300 px haben müssen.

In [8]:
# die Erkkennungslabel müssen angelegt werden. (z.B. 1: 'random' etc.)
classNames = { 0: 'face' }            
    
# Wir laden das Caffe oder Tensorflow Modell
net = cv2.dnn.readNetFromCaffe('resnet10/deploy.prototxt', 'resnet10/res10_300x300_ssd_iter_140000.caffemodel')
    

### 3.3 - Gstreamer Pipeline definieren

Die Initialisierung der Pipeline benötigt, wie oben besprochen weitere Module. Um den Videostream einzulesen werden zwei weitere Konvertierungen, in das Format _NV12_ und von dort nach _RGB_ vorgenommen. Weitere Informationen hierzu:
https://gstreamer.freedesktop.org/documentation/additional/design/mediatype-video-raw.html?gi-language=c#page-description

In [11]:
def receive():
    gst_str = ('udpsrc port=5000 caps = "application/x-rtp, media=(string)video, clock-rate=(int)90000, '
               'encoding-name=(string)RAW, sampling=YCbCr-4:2:0,depth=(string)8,colorimetry=(string)BT601-5, '
               'width=(string)720, height=(string)576, payload=(int)96, a-framerate=10" ! '
               'rtpvrawdepay ! videoconvert ! '
               'video/x-raw, format=(string)NV12! '
               'videoconvert ! appsink emit-signals=True')
    return  WebcamVideoStream(gst_str, cv2.CAP_GSTREAMER).start()# mit Threading
    #return cv2.VideoCapture(gst_str, cv2.CAP_GSTREAMER)  # ohne Threading



Wir passen das zu öffnende Fenster ein wenig an.

In [12]:
WINDOW_NAME = 'CameraDemo'
    
def open_window():
    cv2.startWindowThread()
    cv2.namedWindow(WINDOW_NAME, cv2.WINDOW_NORMAL)
    cv2.resizeWindow(WINDOW_NAME, 720, 576)
    cv2.moveWindow(WINDOW_NAME, 0, 0)
    cv2.setWindowTitle(WINDOW_NAME, 'RTSP-Camera Demo for Xavier AGX')


Um Das Video mit dem DNN zu untersuchen muss es in eine (Bild)Datei mit 300px x 300px umgewandelt werden
https://www.pyimagesearch.com/2017/11/06/deep-learning-opencvs-blobfromimage-works/
Ergebnisse mit mehr als 20% Korrelation werden durch ein Rechteck markiert.

In [ ]:
def read_cam(cap):
    while True:
       
        img = cap.read() # Folgebild von Kamera lesen
        frame = img

        # Abmessungen auslesen und in einen "Blob" umwandeln
        (h, w) = frame.shape[:2]
        blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0,
            (300, 300), (104.0, 177.0, 123.0))

        # Den Blob durchs DNN analysieren lassen und die Vorhersagen in detections 
        # abspeichern
        net.setInput(blob)
        detections = net.forward()
        
        for i in range(0, detections.shape[2]):
            # Die Wahrscheinlichkeit der Vorhersagen iterativ abfragen.
            confidence = detections[0, 0, i, 2]
            # schwache Vorhersagen (mit einem Wert unter 20%) filtern
            if confidence < 0.2:
                continue
            # (x, y)- Koordinaten zur Berechnung eines Rechtecks nutzen
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

            # Umgranzungskästen zeichen und die Wahrscheinlichkeit als Text 
            # oben drüber schreiben
            text = "{:.2f}%".format(confidence * 100)
            y = startY - 10 if startY - 10 > 10 else startY + 10
            cv2.rectangle(frame, (startX, startY), (endX, endY),
                (0, 0, 255), 2)
            cv2.putText(frame, text, (startX, y),
                cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)
        # Fenster öffnen 
        cv2.imshow(WINDOW_NAME, frame)

def main():
    # Verbindung öffnen
    cap = receive()
    # Fenster öffnen
    open_window()
    # Kamerastream einlesen und analysieren
    read_cam(cap)
    
main()

In der Funktion `recieve()` wurde eine OpenCV Funktion ersetzt. Der zugehörige Code steht im folgenden Feld. Hiermit wird ein eigener Thread für den Videostream geöffnet. Bei der Arbeit mit mehreren Videostreams ist diese Ersetzung essentiell.

In [ ]:
# Code for threaded videostreams by Adrian Rosebrock
from threading import Thread
import cv2
class WebcamVideoStream:
    def __init__(self, src=0):
        # initialize the video camera stream and read the first frame
        # from the stream
        self.stream = cv2.VideoCapture(src)
        (self.grabbed, self.frame) = self.stream.read()
        # initialize the variable used to indicate if the thread should
        # be stopped
        self.stopped = False
        
    def start(self):
        # start the thread to read frames from the video stream
        Thread(target=self.update, args=()).start()
        return self
    def update(self):
    #keep looping infinitely until the thread is stopped
        while True:
            # if the thread indicator variable is set, stop the thread
            if self.stopped:
                return
            # otherwise, read the next frame from the stream
            (self.grabbed, self.frame) = self.stream.read()
    def read(self):
        # return the frame most recently read
        return self.frame
    def stop(self):
        # indicate that the thread should be stopped
        self.stopped = True

Das Ergenbis zeigt die Gesichtserkennung im Fenster. Der genutzte Facenet-Graph ist zwar schnell, aber noch nicht besonders zuverlässig.
<img src="pics/Face.png" style="width:500px;">

## 4 - Nutzung von GstInference & weiteren Modulen von RidgeRun
RidgeRun bietet für Gstreamer weitere fortgeschrittene PlugIns an, mit denen sich efiizient dynamische Pipelines mit gleicher Funktion wie dem oberen Beispiel erstllen lassen.
https://developer.ridgerun.com/wiki/index.php?title=Main_Page
Diese Methode hat den Vorteil dass keine Kopie auf der Festplatte benötigt wird.

Für diesen Vortrag nutzen wir Gst-Inference
https://developer.ridgerun.com/wiki/index.php?title=GstInference/Getting_started/Building_the_plugin

Dieses Tol benötigt zur Implementierung der verschiedenen DNN-Protokolle und Architekturen r²Inference.

https://developer.ridgerun.com/wiki/index.php?title=R2Inference/Getting_started/Building_the_library

### 4.1 - Bewertung eines Kamerastream durch DNN mithilfe von GstInference
Damit das untere Bash-Skript funktioniert muss eine USB-Kamera angschlossen sein, die 1280px x 720px Wiedergabe  unterstützt. Um den Einstieg (in RidgeRun Module) zu erleichtern wird eine GUI angeboten
https://developer.ridgerun.com/wiki/index.php?title=GstInference/Example_pipelines_with_hierarchical_metadata

In [ ]:
%%bash
# gstreamer TinyYolo GstInference
CAMERA='/dev/video0'
MODEL_LOCATION='TinyYoloV2_TensorFlow/graph_tinyyolov2_tensorflow.pb'
INPUT_LAYER='input/Placeholder'
OUTPUT_LAYER='add_8'
LABELS='TinyYoloV2_TensorFlow/labels.txt'
gst-launch-1.0 v4l2src device=$CAMERA ! "video/x-raw, width=1280, height=720" ! tee name=t \
t. ! videoconvert ! videoscale ! queue ! net.sink_model \
t. ! queue ! net.sink_bypass \
tinyyolov2 name=net model-location=$MODEL_LOCATION backend=tensorflow backend::input-layer=$INPUT_LAYER backend::output-layer=$OUTPUT_LAYER \
net.src_bypass ! videoconvert ! detectionoverlay labels="$(cat $LABELS)" font-scale=1 thickness=2 ! videoconvert ! xvimagesink sync=false

Das Bashskript öffnet ein Fenster. Zumindest das genutzte Framework ist dabei um einiges schneller als die OpenCV Lösung. Es verarbeitet die doppelte Auflösung. (Genaue Aussagen über die Effektivität kann ich jedoch nicht treffen.)
<img src="pics/Yolo.png" style="width:500px;">

Weitere erwähnenswerte Frameworks mit fortgeschrittenen Funktionen sind **Deepstream** von Nvidia.
https://developer.nvidia.com/deepstream-sdk

Sowie der NNStreamer, ehemals von Samsung, der sogar Tizen OS unterstützt.
https://github.com/nnstreamer/nnstreamer

### 4.2 - GstInterpipe und Gstreamer-Daemon zur Erstellung komplexer Pipelines
<img src="pics/Gstinterpipe-simple-diag.png" style="width:1000px;">

GstInterpipe erlaubt die Kommunikation zweiwe Pipelines. Es beinhaltet die _interpipesink_ und die _interpipesrc_.

Damit wird es möglich in einer High-Level Anwendung auf die Eigenschaften verschiedener Pipelines zentral, Einfluß zu nehmen. Es garantiert die Kompatibilität der Caps in den genutzten Streams. 

Ebenso vereinfacht der Gstreamer-Daemon den Umgang mit mehreren Pipelines. Durch ihn wird es ausserdem einfacher Events und Pufferanweisungen zu synchronisieren.

Sowie Gst-Interpipe
https://developer.ridgerun.com/wiki/index.php?title=GstInterpipe_-_Building_and_Installation_Guide

und den Gstreamer-Daemon
https://developer.ridgerun.com/wiki/index.php?title=GStreamer_Daemon_-_Building_GStreamer_Daemon

Das Beispiel verdeutlicht die Funktionsweise.

In [ ]:
%%bash 
gst-client

# Create the pipeline
pipeline_create testpipe videotestsrc name=vts ! autovideosink

# Play the pipeline
pipeline_play testpipe

# Change a property
element_set testpipe vts pattern ball

# Stop the pipeline
pipeline_stop testpipe

# Destroy the pipeline 
pipeline_delete testpipe

### 4.3 Bringe alles zusammen
Das folgende Bashskript funktionert leider noch nicht. Wenn funktional werden zwei Kameras geöffnet, eine IP-Cam und eine USB-Cam. Beide Videostreams werden durch ein DNN bewertet und markiert. Beide Videostreams werden auf dem Gerät simultan ausgegeben. Dazu werden alle vorher benannten RidgeRun Module genutzt.

```bash
#!/bin/env bash
echo -e "\n ====== Jetson Xavier Multi-Camera AI Demo ====== \n"
# Configure Demo Variables
ROOM_ID=mvdemo
DOWNSCALE_CAPS='video/x-raw,width=480,height=480,format=I420,pixel-aspect-ratio=1/1'
DOWNSCALE_NVMM_CAPS='video/x-raw(memory:NVMM),width=480,height=480,format=I420,pixel-aspect-ratio=1/1'
TY_MODEL_LOCATION='TinyYoloV2_TensorFlow/graph_tinyyolov2_tensorflow.pb'
TY_INPUT_LAYER='input/Placeholder'
TY_OUTPUT_LAYER='add_8'
TY_LABELS='TinyYoloV2_TensorFlow/labels.txt'
TY_OUT_CAPS='video/x-raw,width=1280,height=720,format=RGBA'
INC_MODEL_LOCATION='/InceptionV1_TensorFlow/graph_inceptionv1_tensorflow.pb'
INC_INPUT_LAYER='input'
INC_OUTPUT_LAYER='InceptionV1/Logits/Predictions/Reshape_1'
INC_LABELS='InceptionV1_TensorFlow/imagenet_labels.txt'
INC_OUT_CAPS='video/x-raw,width=720,height=576,format=BGRx'
END_MSJ="!!! Jetson Xavier Multi-Camera AI Demo Finished !!!"

# Camera_1 Pipeline: InceptionV1 Classification Inference applied with GstInference plugin using GPU accelerated TensorFlow.

CAM1="udpsrc port=5000 ! 'application/x-rtp, media=(string)video, clock-rate=(int)90000, encoding-name=(string)RAW, sampling=(string)YCbCr-4:2:0, depth=(string)8, colorimetry=(string)BT601-5, width=(string)720, height=(string)576, payload=(int)96, a-framerate=(string)10' ! rtpvrawdepay ! \
tee name=tee_1 tee_1. ! queue max-size-buffers=1 leaky=downstream ! nvvidconv ! video/x-raw ! inc_1.sink_bypass tee_1. ! \
queue max-size-buffers=1 leaky=downstream ! nvvidconv ! inc_1.sink_model inceptionv1 name=inc_1 backend=tensorflow \
model-location=$INC_MODEL_LOCATION backend::input-layer=$INC_INPUT_LAYER backend::output-layer=$INC_OUTPUT_LAYER inc_1.src_bypass ! \
queue max-size-buffers=1 leaky=downstream ! $INC_OUT_CAPS ! classificationoverlay labels="$(cat $INC_LABELS)" font-scale=4 thickness=4 ! \
queue max-size-buffers=1 leaky=downstream ! nvvidconv ! $DOWNSCALE_NVMM_CAPS ! nvvidconv ! $DOWNSCALE_CAPS ! \
queue max-size-buffers=1 leaky=downstream ! interpipesink name=psink_cam1 caps=$DOWNSCALE_CAPS sync=false qos=false async=false \
enable-last-sample=false drop=true max-buffers=2 forward-eos=false forward-events=false\
"

# Camera_2 Pipeline: TinyYoloV2 Detection Inference applied with GstInference plugin using GPU accelerated TensorFlow.
CAM2="v4l2src device=/dev/video0 ! video/x-raw, width=1280, height=720 ! \
tee name=tee_2 tee_2. ! queue max-size-buffers=1 leaky=downstream ! nvvidconv ! video/x-raw ! ty_2.sink_bypass tee_2. ! \
queue max-size-buffers=1 leaky=downstream ! nvvidconv ! ty_2.sink_model tinyyolov2 name=ty_2 backend=tensorflow \
model-location=$TY_MODEL_LOCATION backend::input-layer=$TY_INPUT_LAYER backend::output-layer=$TY_OUTPUT_LAYER ty_2.src_bypass ! \
queue max-size-buffers=1 leaky=downstream ! detectionoverlay labels="$(cat $TY_LABELS)" font-scale=4 thickness=4 ! $TY_OUT_CAPS ! \
queue max-size-buffers=1 leaky=downstream ! nvvidconv ! $DOWNSCALE_NVMM_CAPS ! nvvidconv ! $DOWNSCALE_CAPS ! \
queue max-size-buffers=1 leaky=downstream ! interpipesink name=psink_cam2 caps=$DOWNSCALE_CAPS sync=false qos=false async=false \
enable-last-sample=false drop=true max-buffers=2 forward-eos=false forward-events=false\
"

# Display Grid Pipeline
GRID="interpipesrc name=grid_cam1 listen-to=psink_cam1 format=time max-bytes=0 block=false max-latency=0 is-live=true allow-renegotiation=false \
enable-sync=true accept-events=false accept-eos-event=false ! queue ! \
perf name=perf1 ! textoverlay name=textoverlay1 text=Classification-Inference-camera_1 shaded-background=true ! mixer.sink_0 \
interpipesrc name=grid_cam2 listen-to=psink_cam2 format=time max-bytes=0 block=false max-latency=0 is-live=true allow-renegotiation=false \
enable-sync=true accept-events=false accept-eos-event=false ! queue ! \
perf name=perf2 ! textoverlay name=textoverlay2 text=Detection-Inference-camera_2 shaded-background=true ! mixer.sink_1 \
videomixer \
name=mixer sink_0::xpos=0 sink_0::ypos=0 sink_1::xpos=577 sink_1::ypos=0  \
queue max-size-buffers=1 leaky=downstream ! videoconvert ! queue max-size-buffers=1 leaky=downstream ! \
nvvidconv ! nvoverlaysink enable-last-sample=false sync=false async=false\
"

# Multi-Stream Pipeline
GST_MULTI_STREAM="interpipesrc name=multivid listen-to=psink_cam1 format=time max-bytes=0 block=false max-latency=0 is-live=true allow-renegotiation=true \
enable-sync=true accept-events=false accept-eos-event=false ! videoscale ! videoconvert ! autovideosink\
"

echo -e "\n ====> Creating Pipelines \n"
gstd-client pipeline_create camera1 $CAM1 &> log
gstd-client pipeline_create camera2 $CAM2 &> log
gstd-client pipeline_create grid $GRID &> log
gstd-client pipeline_create MultiVid $GST_MULTI_STREAM &> log

# Change pipelines to PLAYING STATE
echo -e "\n ====> Starting Pipelines \n"
gstd-client pipeline_play camera1 &> log
gstd-client pipeline_play camera2 &> log
gstd-client pipeline_play grid &> log
gstd-client pipeline_play MultiVid &> log

echo -e "\n ====> Multi-Videostream playing! \n"
```

Am Ende dieses Projekt steht das Verständnis zur Nutzung mehrerer Kameras im Direktanschluss und als IP-Kamera. Und die Auswertung der Videodaten durch ein DNN auf einem mobilen Gerät. Ich hoffe, dass dieser Ansatzpunkt verständlich und umfassend genug herübergebracht wurde. Für weitere Fragen und natürlich Anregungen bin ich weiterhin erreichbar.